# Práctica 4

### Pasos de este análisis
1. Carga de datos
1. Exploración
1. Procesamiento


# Analisis de datos del titanic

## 1. Cargando datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# lectura de datos en Python
train = pd.read_csv('./../data/train.csv')
test = pd.read_csv('./../data/test.csv')

In [ ]:
train.head()

## 2. Explorando datos

In [ ]:
# que columnas tienen los datos?
train.columns

In [ ]:
# qué tamaño tienen los datos?
train.shape

In [ ]:
# hay valores nulos en los datos?
train.info()

In [ ]:
# como se distribuyen las variables numéricas
train.describe()

In [ ]:
# como se comportan las variables categóricas
train.describe(include=['O'])

## 2.1 EDA: Estudio de variable target

In [ ]:
train.groupby(['Survived']).count()['PassengerId']

In [ ]:
# target vs sex
train.groupby(['Survived','Sex']).count()['PassengerId']

In [ ]:
grouped_sex = train.groupby(['Survived','Sex']).count()['PassengerId']
print(grouped_sex)
(grouped_sex.unstack(level=0).plot.bar())
plt.show()

In [ ]:
# tarea hacer para otras variables

In [ ]:
# embarked vs pclass
print(train.groupby(['Pclass', 'Embarked'])
        .count()['PassengerId']
        .unstack(level=0)
        .plot.bar())

## 3.Procesamiento de datos

Empezamos seleccionando las variables que queremos trabajar que serían: 
* Survived
* Sex
* Age
* Pclass

In [ ]:
train[['Survived', 'Sex', 'Age', 'Pclass']].head(3)

Estudiamos los datos nulos

In [ ]:
train[['Survived', 'Sex', 'Age', 'Pclass']].info()

-----------------------
Tenemos que mejorar

*    Datos faltantes en Age `train['Age'].isna()`
*    La variable Sex aparece como object y queremos int o float para algoritmos
-----------------------

In [ ]:
# mirar como se distribuyen los nulos en edad
(train[train['Age'].isna()]
      .groupby(['Sex', 'Pclass'])
      .count()['PassengerId']
      .unstack(level=0))

In [ ]:
(train[train['Age'].isna()]
      .groupby(['SibSp', 'Parch'])
      .count()['PassengerId']
      .unstack(level=0))

De arriba se puede concluir que era gente que viajaba mayormente sola y la mayoría eran de 3era clase.

Esto nos da la idea de que se puede crear una variable que indique si la persona viajaba sola o acompañada.

La crearemos más adelante


In [ ]:
# calcular mediana de Age para imputar
train['Age'].median()

In [ ]:
# imputar valor para rellenar nulos
train['Age'] = train['Age'].fillna(28.0)
train[['Survived', 'Sex', 'Age', 'Pclass']].info()

Ya no tenemos nulos. Falta resolver lo de pasar Sex a int

In [ ]:
# map para label encoding
train['Sex'] = train['Sex'].map({'female': 1, 'male': 0}).astype(int)

Ahora tenemos la tabla preprocesada lista

In [ ]:
train[['Survived', 'Sex', 'Age', 'Pclass']].head(3)

## 3.1 Crear nuevas variables

In [ ]:
# crear nueva variable tipo flag "solo"
train['FlagSolo'] = np.where(
    (train['SibSp'] == 0) & (train['Parch'] == 0), 1, 0)

In [ ]:
grouped_flag = train.groupby(['Survived','FlagSolo']).count()['PassengerId']
print(grouped_flag)
(grouped_flag.unstack(level=0).plot.bar())
plt.show()

In [ ]:
train[['Survived', 'Sex', 'Age', 'Pclass', 'FlagSolo']].head(3)

Estos ya serían los datos con los que se podrían hacer modelos. En las tablas que encontramos dentro de la carpeta notebooks y data, vemos cómo quedaría la interpretación de estos datos según los aspectos y variables que hemos modificado. A esto también se le podría añadir la funcionalidad de test, pero es un proceso complejo que he preferido no incluir.

In [ ]:
# variable dependiente
Y_train = train['Survived']

# preprocesamiento de variables independientes
features = ['Sex', 'Age', 'Pclass', 'FlagSolo']
X_train = train[features]

print(Y_train.shape, X_train.shape)